# Sample Code

## MobileNetV2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten,
GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
# Define dataset path
dataset_path = "/content/drive/MyDrive/Vehicles"
# Preprocessing using ImageDataGenerator
datagen = ImageDataGenerator(
 rescale=1.0/255.0,
 validation_split=0.2, # 20% validation split
)
# Training and validation data generators
train_generator = datagen.flow_from_directory(
 dataset_path,
 target_size=(224, 224),
 batch_size=32,
 class_mode='categorical',
 subset='training',
)
validation_generator = datagen.flow_from_directory(
 dataset_path,
 target_size=(224, 224),
 batch_size=32,
 class_mode='categorical',
 subset='validation',
)
# Get class indices
class_labels = train_generator.class_indices
print("Class Labels:", class_labels)
# Load pre-trained MobileNetV2 model (exclude the top layer)
base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False,
weights='imagenet')
# Freeze the base model
base_model.trainable = False
# Add custom classification head
model = Sequential([
 base_model,
 GlobalAveragePooling2D(),
 Dense(128, activation='relu'),
 Dropout(0.5),
 Dense(len(class_labels), activation='softmax') # Output layer with the number of
classes
])
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()
# Train the model
history = model.fit(
 train_generator,
 validation_data=validation_generator,
 epochs=10,
 steps_per_epoch=train_generator.samples // train_generator.batch_size,
 validation_steps=validation_generator.samples // validation_generator.batch_size
)
# Evaluate the model
validation_generator.reset()
loss, accuracy = model.evaluate(validation_generator)
print(f"Validation Loss: {loss:.4f}, Validation Accuracy: {accuracy:.4f}")
# Classification report
validation_generator.reset()
predictions = model.predict(validation_generator)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = validation_generator.classes
class_labels = list(validation_generator.class_indices.keys())
print("Classification Report:")
print(classification_report(true_classes, predicted_classes,
target_names=class_labels))
# Confusion Matrix
conf_matrix = confusion_matrix(true_classes, predicted_classes)
plt.figure(figsize=(10, 8))
plt.imshow(conf_matrix, interpolation='nearest', cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.colorbar()
plt.show()
# Unfreeze some layers in the base model
base_model.trainable = True
# Compile the model with a lower learning rate
model.compile(optimizer=Adam(learning_rate=1e-5),
loss='categorical_crossentropy', metrics=['accuracy'])
# Fine-tune the model
history_fine_tune = model.fit(
 train_generator,
 validation_data=validation_generator,
 epochs=5,
 steps_per_epoch=train_generator.samples // train_generator.batch_size,
 validation_steps=validation_generator.samples // validation_generator.batch_size
)

## VGG19

In [ ]:
import os
import shutil
import random
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torchvision import models
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder

# Function to split dataset into training and validation sets
def split_dataset(dataset_dir, output_dir, train_ratio=0.8):
    """
    Splits a dataset into train and validation sets.
    Args:
        dataset_dir (str): Path to the original dataset directory.
        output_dir (str): Path to the output directory for train and validation splits.
        train_ratio (float): Ratio of training data. Default is 0.8 (80% training, 20% validation).
    """
    train_dir = os.path.join(output_dir, 'train')
    val_dir = os.path.join(output_dir, 'val')
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(val_dir, exist_ok=True)

    # Iterate over each class folder
    for class_name in os.listdir(dataset_dir):
        class_dir = os.path.join(dataset_dir, class_name)
        if not os.path.isdir(class_dir):
            continue

        # Create class subdirectories in train and val folders
        os.makedirs(os.path.join(train_dir, class_name), exist_ok=True)
        os.makedirs(os.path.join(val_dir, class_name), exist_ok=True)

        # Get all file names in the class directory
        files = os.listdir(class_dir)
        random.shuffle(files)

        # Split files into train and validation sets
        split_idx = int(len(files) * train_ratio)
        train_files = files[:split_idx]
        val_files = files[split_idx:]

        # Move files to respective directories
        for file_name in train_files:
            src = os.path.join(class_dir, file_name)
            dst = os.path.join(train_dir, class_name, file_name)
            shutil.copy2(src, dst)

        for file_name in val_files:
            src = os.path.join(class_dir, file_name)
            dst = os.path.join(val_dir, class_name, file_name)
            shutil.copy2(src, dst)

    print(f"Dataset split completed. Train and val sets are saved in {output_dir}")

# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Split the dataset (adjust paths as needed)
original_dataset_path = "/content/drive/MyDrive/Vehicle Classification/Vehicles"  # Replace with your original dataset path
split_output_path = "/content/drive/MyDrive/vgg19/split output "         # Replace with the path to save the split dataset
split_dataset(original_dataset_path, split_output_path)

# Define transformations for the dataset
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize for VGG-19
])

# Load train and validation datasets
train_dataset = ImageFolder(root=os.path.join(split_output_path, "train"), transform=transform)
val_dataset = ImageFolder(root=os.path.join(split_output_path, "val"), transform=transform)

# Create data loaders
train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=32, shuffle=False)

# Load the VGG-19 model
vgg19 = models.vgg19(pretrained=True)

# Modify the final layer to match the number of vehicle classes
num_classes = len(train_dataset.classes)  # Number of vehicle categories
vgg19.classifier[6] = nn.Linear(vgg19.classifier[6].in_features, num_classes)

# Move the model to the device
vgg19 = vgg19.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(vgg19.parameters(), lr=0.001)

# Training the model
epochs = 10
for epoch in range(epochs):
    vgg19.train()
    running_loss = 0.0
    for i, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = vgg19(images)
        loss = criterion(outputs, labels)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if (i + 1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')

# Evaluation
vgg19.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = vgg19(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Validation Accuracy: {100 * correct / total:.2f}%')

# Save the model
torch.save(vgg19.state_dict(), 'vgg19_vehicle_classification.pth')

# Class Names
print(f"Classes: {train_dataset.classes}")

from sklearn.metrics import classification_report

# Function to generate classification report
def generate_classification_report(model, data_loader, class_names, device):
    """
    Generates a classification report for the given model and data loader.

    Args:
        model: Trained model.
        data_loader: DataLoader for validation or test dataset.
        class_names: List of class names.
        device: Device to perform computations on.

    Returns:
        Classification report as a string.
    """
    model.eval()  # Set model to evaluation mode
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in data_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Generate classification report
    report = classification_report(all_labels, all_preds, target_names=class_names)
    print("Classification Report:")
    print(report)
    return report

# Generate and print classification report
classification_report_str = generate_classification_report(vgg19, val_loader, train_dataset.classes, device)

## ResNet50

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
from torchvision import models
import os
import shutil
import random
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, f1_score, classification_report
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix  # Import confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


# Function to split dataset into train and validation sets
def split_dataset(dataset_dir, output_dir, train_ratio=0.8):
    train_dir = os.path.join(output_dir, 'train')
    val_dir = os.path.join(output_dir, 'val')
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(val_dir, exist_ok=True)

    for class_name in os.listdir(dataset_dir):
        class_dir = os.path.join(dataset_dir, class_name)
        if not os.path.isdir(class_dir):
            continue

        os.makedirs(os.path.join(train_dir, class_name), exist_ok=True)
        os.makedirs(os.path.join(val_dir, class_name), exist_ok=True)

        files = os.listdir(class_dir)
        random.shuffle(files)

        split_idx = int(len(files) * train_ratio)
        train_files = files[:split_idx]
        val_files = files[split_idx:]

        for file_name in train_files:
            shutil.copy2(os.path.join(class_dir, file_name), os.path.join(train_dir, class_name, file_name))

        for file_name in val_files:
            shutil.copy2(os.path.join(class_dir, file_name), os.path.join(val_dir, class_name, file_name))

    print(f"Dataset split completed. Train and val sets are saved in {output_dir}")

# Dataset transformation
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Split the dataset (replace with actual dataset paths)
split_dataset('/content/drive/MyDrive/Vehicle Classification/Vehicles', '/content/drive/MyDrive/ResNet50/spilt data')

# Load datasets
train_dir = '/content/drive/MyDrive/ResNet50/spilt data/train'
val_dir = '/content/drive/MyDrive/ResNet50/spilt data/val'

train_dataset = torchvision.datasets.ImageFolder(train_dir, transform=transform)
val_dataset = torchvision.datasets.ImageFolder(val_dir, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=False)

# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load ResNet-50 model
resnet50 = models.resnet50(pretrained=True)

# Modify the final layer for the number of classes in your dataset
num_classes = len(train_dataset.classes)
resnet50.fc = nn.Linear(resnet50.fc.in_features, num_classes)

# Move model to device
resnet50 = resnet50.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(resnet50.parameters(), lr=0.001)

# Training loop
epochs = 10
for epoch in range(epochs):
    resnet50.train()
    running_loss = 0.0
    for i, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = resnet50(images)
        loss = criterion(outputs, labels)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}')

# Save the model
torch.save(resnet50.state_dict(), 'resnet50_vehicle_classification.pth')

# Evaluation function
def evaluate_model(model, data_loader, class_names):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in data_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    return all_labels, all_preds

# Generate confusion matrix
def plot_confusion_matrix(all_labels, all_preds, class_names):
    cm = confusion_matrix(all_labels, all_preds)
    plt.figure(figsize=(10, 8))
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
    disp.plot(cmap=plt.cm.Blues, xticks_rotation=45, ax=plt.gca())
    plt.title("Confusion Matrix")
    plt.show()

# Generate F1 score
def compute_f1_score(all_labels, all_preds):
    f1_macro = f1_score(all_labels, all_preds, average='macro')
    f1_weighted = f1_score(all_labels, all_preds, average='weighted')
    print(f"F1 Score (Macro): {f1_macro:.4f}")
    print(f"F1 Score (Weighted): {f1_weighted:.4f}")

# Generate classification report
def generate_classification_report(all_labels, all_preds, class_names):
    report = classification_report(all_labels, all_preds, target_names=class_names)
    print("Classification Report:")
    print(report)

# Evaluate and generate metrics
all_labels, all_preds = evaluate_model(resnet50, val_loader, train_dataset.classes)
plot_confusion_matrix(all_labels, all_preds, train_dataset.classes)
compute_f1_score(all_labels, all_preds)
generate_classification_report(all_labels, all_preds, train_dataset.classes)

## ResNet18

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split

# Define data transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # ResNet requires 224x224 input
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load the dataset (replace with your dataset path)
dataset = datasets.ImageFolder(root='/content/drive/MyDrive/Vehicle Classification/Vehicles', transform=transform)

# Split the dataset into training and validation sets
train_size = int(0.8 * len(dataset))  # 80% for training
val_size = len(dataset) - train_size  # 20% for validation
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Load pre-trained ResNet model
resnet = models.resnet18(pretrained=True)  # Use ResNet-18, you can choose ResNet-50, etc.

# Modify the final layer for 4-class classification
num_features = resnet.fc.in_features
resnet.fc = nn.Linear(num_features, 4)

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet = resnet.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet.parameters(), lr=0.001)

# Training loop
def train(model, dataloader, criterion, optimizer):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in dataloader:
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Statistics
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    return running_loss / len(dataloader), correct / total

# Validation loop
def validate(model, dataloader, criterion):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Statistics
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return running_loss / len(dataloader), correct / total

# Training the model
epochs = 10
for epoch in range(epochs):
    train_loss, train_acc = train(resnet, train_loader, criterion, optimizer)
    val_loss, val_acc = validate(resnet, val_loader, criterion)

    print(f"Epoch {epoch+1}/{epochs}:")
    print(f"  Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.4f}")
    print(f"  Val Loss: {val_loss:.4f}, Val Accuracy: {val_acc:.4f}")

# Save the trained model
torch.save(resnet.state_dict(), 'vehicle_classification_resnet.pth')

from sklearn.metrics import classification_report  # Import classification_report

# ... (Your existing training code) ...

# Get predictions for the entire dataset (train and validation)
all_labels = []
all_predictions = []

# Assuming you want to get the predictions for your validation set
# You might want to iterate through the entire dataset or a specific split like val_loader
for inputs, labels in val_loader:
    inputs, labels = inputs.to(device), labels.to(device)
    outputs = resnet(inputs)
    _, predicted = torch.max(outputs, 1)
    all_labels.extend(labels.cpu().numpy())  # Move labels to CPU and convert to numpy
    all_predictions.extend(predicted.cpu().numpy())  # Move predictions to CPU and convert to numpy

# Generate the classification report
report = classification_report(all_labels, all_predictions, target_names=val_loader.dataset.dataset.classes)  # Use correct loader
print(report) # Corrected indentation

!pip install scikit-learn  # Install scikit-learn if you haven't already

# Get predictions for the entire validation set
all_labels = []
all_predictions = []
for inputs, labels in val_loader:
    inputs, labels = inputs.to(device), labels.to(device)
    outputs = resnet(inputs)
    _, predicted = torch.max(outputs, 1)
    all_labels.extend(labels.cpu().numpy())
    all_predictions.extend(predicted.cpu().numpy())

# Generate the classification report
report = classification_report(all_labels, all_predictions, target_names=val_loader.dataset.dataset.classes)
print(report)

# Generate the confusion matrix
cm = confusion_matrix(all_labels, all_predictions)

# Plot the confusion matrix using seaborn
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=val_loader.dataset.dataset.classes,
            yticklabels=val_loader.dataset.dataset.classes)
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()